In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
import math

from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Lambda, Embedding, Flatten

2024-10-19 21:45:54.807387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
file = h5py.File('xy.h5', 'r')

x = file['x'][:]
y = file['x'][:]

x.shape, y.shape

((50000, 16384), (50000, 16384))

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, x, test_size=0.2, random_state=42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((40000, 16384), (40000, 16384), (10000, 16384), (10000, 16384))

In [4]:
batch_size = 4096

def generate_training():
    while True:
        indices = np.random.choice(len(x_train), batch_size)
        yield x_train[indices], y_train[indices]

def generate_test():
    while True:
        indices = np.random.choice(len(x_test), batch_size)
        yield x_test[indices], y_test[indices]

train_generator = generate_training()
test_generator = generate_test()

In [5]:
model = Sequential([
    Input(shape=(16384,)),
    Dense(128, activation='relu'),
    Dense(16384, activation='relu')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2097280   
                                                                 
 dense_1 (Dense)             (None, 16384)             2113536   
                                                                 
Total params: 4210816 (16.06 MB)
Trainable params: 4210816 (16.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
optimizer = Adam(learning_rate=.1)
model.compile(optimizer='adam', loss='mae', metrics=['mse'])
history = model.fit(train_generator, epochs=32, steps_per_epoch=len(x_train)/batch_size, validation_data=test_generator, validation_steps=len(x_test)/batch_size)

Epoch 1/32
9/9 [==============================] - 11s 1s/step - loss: 2.4409 - mse: 81.7605 - val_loss: 2.1570 - val_mse: 77.8808
Epoch 2/32
9/9 [==============================] - 10s 1s/step - loss: 2.0688 - mse: 75.2151 - val_loss: 1.9816 - val_mse: 72.0790
Epoch 3/32
9/9 [==============================] - 11s 1s/step - loss: 1.8623 - mse: 67.2572 - val_loss: 1.7864 - val_mse: 61.8483
Epoch 4/32
9/9 [==============================] - 11s 1s/step - loss: 1.7500 - mse: 59.1338 - val_loss: 1.7479 - val_mse: 60.1668
Epoch 5/32
9/9 [==============================] - 11s 1s/step - loss: 1.6838 - mse: 58.8803 - val_loss: 1.6804 - val_mse: 59.5305
Epoch 6/32
9/9 [==============================] - 11s 1s/step - loss: 1.6385 - mse: 57.6590 - val_loss: 1.6146 - val_mse: 55.6762
Epoch 7/32
9/9 [==============================] - 10s 1s/step - loss: 1.5803 - mse: 54.6171 - val_loss: 1.5586 - val_mse: 53.8791
Epoch 8/32
9/9 [==============================] - 10s 1s/step - loss: 1.5277 - mse: 52.298

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'], label='Train Accuracy')
plt.plot(history.history['val_loss'], label='Val Accuracy')
 
plt.show()